In [1]:
import pandas as pd
import numpy as np
import os
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, FloatType
from pyspark.sql import Row

2023-10-02 01:23:38.766395: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-02 01:23:38.766456: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-02 01:23:38.766486: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-02 01:23:38.776200: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-02 01:23:39.971616: W tensorflow/compiler/

# HandLandmarker Detector

In [2]:
# Create an HandLandmarker object.
base_options = python.BaseOptions(model_asset_path='hand_landmarker.task')
options = vision.HandLandmarkerOptions(base_options=base_options,
                                       num_hands=2)
detector = vision.HandLandmarker.create_from_options(options)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


# Data processing

In [3]:
#Create empty DatFrame
df = pd.DataFrame(columns = [f"x{i}" for i in range(21)] + [f"y{i}" for i in range(21)] + [f"z{i}" for i in range(21)])
df

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,z11,z12,z13,z14,z15,z16,z17,z18,z19,z20


In [4]:
# Function to recursively read files in folders
def read_images(folder_path):
    X = []
    y = []
    classes = set([])
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)
            
            file_name = file
            file_path = root
            
            image = mp.Image.create_from_file(file_path + "/" + file_name)
            
            detection_result = detector.detect(image)
            letter = file_path.split("/")[-1]
            if(len(detection_result.hand_landmarks) == 1 and letter != "Blank"):
                X.append(detection_result.hand_landmarks)
                y.append(letter)
                classes.add(letter)
            
    return X, y

In [5]:
X_train, y_train = read_images("./data/Train_Alphabet/")

In [ ]:
X_test, y_test = read_images("./data/Test_Alphabet/")

In [ ]:
def image_to_df(samples, df):
    samples = np.array(samples)
    samples = [sample.flatten() for sample in samples]
    for i in range(len(samples)):
        row = {}
        for j in range(len(samples[i])):
            row[f"x{j}"] = samples[i][j].x
            row[f"y{j}"] = samples[i][j].y
            row[f"z{j}"] = samples[i][j].z
        df.loc[len(df)] = row

In [ ]:
image_to_df(X_train, df)
image_to_df(X_test, df)

In [ ]:
y = np.append(y_train, y_test, axis=0)
df["class"] = y

NameError: name 'y_test' is not defined

In [ ]:
df

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,z11,z12,z13,z14,z15,z16,z17,z18,z19,z20
0,0.120298,0.187145,0.229388,0.227301,0.201995,0.270536,0.308709,0.277825,0.263552,0.228318,...,-0.041533,-0.043614,0.000357,-0.033487,-0.045169,-0.044333,-0.004517,-0.023201,-0.033710,-0.039308
1,0.780754,0.730897,0.684163,0.644360,0.606403,0.716289,0.701756,0.696184,0.693506,0.750416,...,-0.056988,-0.048875,-0.026401,-0.058895,-0.050627,-0.035684,-0.028742,-0.047905,-0.037866,-0.025401


In [ ]:
df.to_csv("data/asl_dataset.csv")